## API Central Bank of Costa Rica

The idea of this notebook is to extract data from the API of the Central Bank of Costa Rica. With this we can get the data for the model.

In [31]:
import requests
import os
import pandas as pd
import numpy as np
import time
import sys
from pathlib import Path
sys.path.append('../')
from src.data import cbk_data

First we create the parametes for call

In [4]:
url = "https://gee.bccr.fi.cr/Indicadores/Suscripciones/WS/wsindicadoreseconomicos.asmx/ObtenerIndicadoresEconomicos?"
current_time = time.strftime("%d/%m/%Y")
params_bccr = {
    "Indicador": "3181",
    "FechaInicio": "01/01/1999",
    "FechaFinal": current_time,
    "Nombre": "Marlon",
    "Subniveles": "S",
    "CorreoElectronico": os.getenv("BCCR_USER"),
    "Token": os.getenv("BCCR_PASS")
}

code_vars = {
            "exp" : "1992",
            "imp" : "1993",
            "imae": "87703",
            "tpm": "3541",
            "monex_amount_mean" : "3451",
            "tc_monex" : "3438",
            "gold" : "1979",
            "tc_sell" : "3140",
            "tc_buy" : "3280",
            # "reserv" : "19578",
            "mil_count" : "3799",
            # "mil_lapse" : "3801"
            # "deuda" : "3745"
             }


We do the calls

In [5]:
api_varscall = {}
for k, v in code_vars.items():
  api_varscall[k] = cbk_data.api_call(var_code= v, start_date = "01/01/1999",
                             end_date = current_time, type_exc = "", var_name = k)
  api_varscall[k].params_bccr()
  api_varscall[k].api_call()
  api_varscall[k] = api_varscall[k].create_data()
  api_varscall[k] = api_varscall[k].drop(columns='type').rename(columns={'bank' : 'variable'})

We need to check the numbers of codes behind the call, because as we set the params with subhead codes, it would be some other codes that we don't want

In [24]:
for k, v in code_vars.items():
    current_dt = api_varscall[k]
    lastav_year = np.max(pd.DatetimeIndex(current_dt.loc[:, "date"]).year)
    print(f"The var is {k}")
    print(f"The numbers of rows are {current_dt.shape[0]}")
    print(f"The numbers of codes are {len(api_varscall[k].loc[:, 'cod'].unique())}")
    print(f"The las available year is: {lastav_year}")

The var is exp
The numbers of rows are 283
The numbers of codes are 1
The las available year is: 2022
The var is imp
The numbers of rows are 283
The numbers of codes are 1
The las available year is: 2022
The var is imae
The numbers of rows are 13237
The numbers of codes are 47
The las available year is: 2022
The var is tpm
The numbers of rows are 5756
The numbers of codes are 1
The las available year is: 2022
The var is monex_amount_mean
The numbers of rows are 5379
The numbers of codes are 1
The las available year is: 2022
The var is tc_monex
The numbers of rows are 5379
The numbers of codes are 1
The las available year is: 2022
The var is gold
The numbers of rows are 6535
The numbers of codes are 1
The las available year is: 2022
The var is tc_sell
The numbers of rows are 5846
The numbers of codes are 1
The las available year is: 2022
The var is tc_buy
The numbers of rows are 5814
The numbers of codes are 1
The las available year is: 2022
The var is mil_count
The numbers of rows are 

There are more codes for imae, so we set only the 87703 code for IMAE

In [27]:
api_varscall["imae"] = api_varscall["imae"].query("cod == 87703")

,cod,date,value,var
0,87703,1999-01-31T00:00:00-06:00,52.197280,imae
1,87703,1999-02-28T00:00:00-06:00,48.747353,imae
2,87703,1999-03-31T00:00:00-06:00,50.386924,imae
3,87703,1999-04-30T00:00:00-06:00,47.016367,imae
4,87703,1999-05-31T00:00:00-06:00,48.617007,imae
...,...,...,...,...
277,87703,2022-02-28T00:00:00-06:00,108.611735,imae
278,87703,2022-03-31T00:00:00-06:00,118.166052,imae
279,87703,2022-04-30T00:00:00-06:00,107.834739,imae
280,87703,2022-05-31T00:00:00-06:00,109.730103,imae


We store the data as an csv in raw

In [36]:
data2save = pd.concat(api_varscall, ignore_index=True)
outpath = Path.cwd().parent.joinpath("data", "raw_cbk.csv")
with outpath as e:
    data2save.to_csv(e, index=False)